In [12]:
# !pip install accelerate
# !pip install transformers[torch]
# !pip install accelerate -U

In [13]:
!pip install transformers

In [90]:
!pip install datasets

In [137]:
from datasets import load_dataset
from transformers import AutoTokenizer
import numpy as np

### pre_process functions

In [138]:

# Labels
label_list = ["B", "I", "O"]
lbl2idx = {"B": 0, "I": 1, "O": 2}
idx2label = {0: "B", 1: "I", 2: "O"}

# Tokenizer
# tokenizer = AutoTokenizer.from_pretrained("bloomberg/KBIR", add_prefix_space=True)
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
max_length = 512

# Dataset parameters
dataset_full_name = "midas/inspec"
dataset_subset = "raw"
dataset_document_column = "document"
dataset_biotags_column = "doc_bio_tags"

def preprocess_fuction(all_samples_per_split):
    tokenized_samples = tokenizer.batch_encode_plus(
        all_samples_per_split[dataset_document_column],
        padding="max_length",
        truncation=True,
        is_split_into_words=True,
        max_length=max_length,
    )
    total_adjusted_labels = []
    for k in range(0, len(tokenized_samples["input_ids"])):
        prev_wid = -1
        word_ids_list = tokenized_samples.word_ids(batch_index=k)
        existing_label_ids = all_samples_per_split[dataset_biotags_column][k]
        i = -1
        adjusted_label_ids = []

        for wid in word_ids_list:
            if wid is None:
                adjusted_label_ids.append(lbl2idx["O"])
            elif wid != prev_wid:
                i = i + 1
                adjusted_label_ids.append(lbl2idx[existing_label_ids[i]])
                prev_wid = wid
            else:
                adjusted_label_ids.append(
                    lbl2idx[
                        f"{'I' if existing_label_ids[i] == 'B' else existing_label_ids[i]}"
                    ]
                )

        total_adjusted_labels.append(adjusted_label_ids)
    tokenized_samples["labels"] = total_adjusted_labels
    return tokenized_samples

# Load dataset
dataset = load_dataset(dataset_full_name, dataset_subset)

# Preprocess dataset
tokenized_dataset = dataset.map(preprocess_fuction, batched=True)



Repo card metadata block was not found. Setting CardData to empty.


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [139]:
################################# POST PROCESS FUNCTION
# # Define post_process functions
# def concat_tokens_by_tag(keyphrases):
#     keyphrase_tokens = []
#     for id, label in keyphrases:
#         if label == "B":
#             keyphrase_tokens.append([id])
#         elif label == "I":
#             if len(keyphrase_tokens) > 0:
#                 keyphrase_tokens[len(keyphrase_tokens) - 1].append(id)
#     return keyphrase_tokens


# def extract_keyphrases(example, predictions, tokenizer, index=0):
#     keyphrases_list = [
#         (id, idx2label[label])
#         for id, label in zip(
#             np.array(example["input_ids"]).squeeze().tolist(), predictions[index]
#         )
#         if idx2label[label] in ["B", "I"]
#     ]

#     processed_keyphrases = concat_tokens_by_tag(keyphrases_list)
#     extracted_kps = tokenizer.batch_decode(
#         processed_keyphrases,
#         skip_special_tokens=True,
#         clean_up_tokenization_spaces=True,
#     )
#     return np.unique([kp.strip() for kp in extracted_kps])


### exploring dataset - preparation

In [140]:
dataset = load_dataset("midas/inspec", 'raw')
tokenized_dataset = dataset.map(preprocess_fuction, batched=True)

Repo card metadata block was not found. Setting CardData to empty.


In [141]:
train_data = tokenized_dataset['train'][:5]
validation_data = tokenized_dataset['validation'][:2]
test_data = tokenized_dataset['test'][:2]

In [142]:
from datasets import Dataset, DatasetDict

# Create Dataset objects for each split
train_dataset = Dataset.from_dict(train_data)
test_dataset = Dataset.from_dict(test_data)
validation_dataset = Dataset.from_dict(validation_data)

# Create a DatasetDict by combining the datasets
dataset_dict = DatasetDict({
    'train': train_dataset,
    'test': test_dataset,
    'validation': validation_dataset
})

# print(dataset_dict) #this is just a subset of tokenized_dataset
# print(tokenized_dataset) #after tokenizing
# print(dataset) # data before tokenizing

### Fine Tuning

In [143]:
# model_name = "ml6team/keyphrase-extraction-kbir-openkp"
model_name = 'ml6team/keyphrase-extraction-distilbert-inspec'

In [144]:
# customized the evaluation metrics
from datasets import load_dataset, ClassLabel
import datasets
from datasets.features import ClassLabel

accuracy = datasets.load_metric("accuracy")
f1_metric = datasets.load_metric("f1")

def my_compute_metrics(eval_pred):
    logits, labels = eval_pred
    refs = labels.flatten()
    predictions = np.argmax(logits, axis=-1)
    # print(logits.shape)
    # print(predictions.shape)
    pred = predictions.flatten()
    return {
        "accuracy": accuracy.compute(predictions=pred, references=refs)["accuracy"],
        "f1_micro": f1_metric.compute(predictions=pred, references=refs, average="micro")["f1"],
        "f1_macro": f1_metric.compute(predictions=pred, references=refs, average="macro")["f1"],
    }


In [145]:
from transformers import AutoModelForTokenClassification, Trainer, TrainingArguments, DataCollatorForTokenClassification
import torch

# TOKENIZATION

'''# Load your dataset (e.g., using datasets.load_dataset())
dataset = datasets.load_dataset('your_dataset_name')

# Tokenize and preprocess the dataset
tokenizer = AutoTokenizer.from_pretrained(model_name)
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
'''

#####

# Load your pretrained model
model = AutoModelForTokenClassification.from_pretrained(model_name)

# Define your training arguments
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/thesis",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=8,
    save_steps=100,
    evaluation_strategy="steps",
    eval_steps=100,
    learning_rate = 1e-4   # missng early_stop training, possible to customize
)

# Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=None,  # If needed, define a data collator
    train_dataset=dataset_dict['train'],
    eval_dataset=dataset_dict['validation'],
    compute_metrics = my_compute_metrics,  # Define a metric function if needed
)

# Finetuning the model
trainer.train()


Step,Training Loss,Validation Loss


TrainOutput(global_step=5, training_loss=0.0012008005753159523, metrics={'train_runtime': 56.0341, 'train_samples_per_second': 0.446, 'train_steps_per_second': 0.089, 'total_flos': 3266386560000.0, 'train_loss': 0.0012008005753159523, 'epoch': 5.0})

In [146]:
#evaluate on the val set
trainer.evaluate()

{'eval_loss': 0.18946310877799988,
 'eval_accuracy': 0.974609375,
 'eval_f1_micro': 0.974609375,
 'eval_f1_macro': 0.7263403364207722,
 'eval_runtime': 1.2857,
 'eval_samples_per_second': 1.556,
 'eval_steps_per_second': 0.778,
 'epoch': 5.0}

In [147]:
#predict on the test data
prediction = trainer.predict(dataset_dict['test'])

In [148]:
'''
# Save the fine-tuned model and tokenizer
model.save_pretrained('fine_tuned_model_directory')
tokenizer.save_pretrained('fine_tuned_model_directory')
'''

"\n# Save the fine-tuned model and tokenizer\nmodel.save_pretrained('fine_tuned_model_directory')\ntokenizer.save_pretrained('fine_tuned_model_directory')\n"

In [149]:
# Get the predicted logits
predicted_logits = prediction.predictions
predicted_labels = np.argmax(predicted_logits, axis=-1)

# Get the true labels or target values
true_labels = prediction.label_ids

#example of the first 5 tokens' ground true labels and their predicted logits
print(predicted_logits[0][:1])
print(predicted_labels[0][100:105])
print(true_labels[0][100:105])

[[-5.542736 -5.177245  8.806656]]
[2 2 1 2 2]
[2 2 2 2 2]


### post_processing

In [150]:
# # Define post_process functions
# def concat_tokens_by_tag(keyphrases):
#     keyphrase_tokens = []
#     for id, label in keyphrases:
#         if label == "B":
#             keyphrase_tokens.append([id])
#         elif label == "I":
#             if len(keyphrase_tokens) > 0:
#                 keyphrase_tokens[len(keyphrase_tokens) - 1].append(id)
#     return keyphrase_tokens


# def extract_keyphrases(example, predicted_labels, tokenizer, index=0):
#     keyphrases_list = [
#         (id, idx2label[label]) #Did try: convert idx2label[label] to a string type, do not work
#         for id, label in zip(
#             np.array(example["input_ids"]).squeeze().tolist(), predicted_labels
#         )
#         if idx2label[label] in ["B", "I"]
#     ]

#     processed_keyphrases = concat_tokens_by_tag(keyphrases_list)
#     extracted_kps = tokenizer.batch_decode(
#         processed_keyphrases,
#         skip_special_tokens=True,
#         clean_up_tokenization_spaces=True,
#     )
#     return np.unique([kp.strip() for kp in extracted_kps])

'''#this function does not work because of the data type of the predicted_labels.
example = dataset_dict['test'][0] #everything
extract_keyphrases(example, predicted_labels, tokenizer, index=0) #extract keyphrases'''

"#this function does not work because of the data type of the predicted_labels.\nexample = dataset_dict['test'][0] #everything\nextract_keyphrases(example, predicted_labels, tokenizer, index=0) #extract keyphrases"